# TODO
- annotate well so someone reading can understand
- figure out matryoshkaMHA
- figure out forward()
- figure out generate()
- copy the cosine similarity visual exploration tools from `matryoshka_embeddings_gpt`
- train & save a model

#### !!!! DO NOT RUN THIS FIRST CELL UNLESS YOU HAVE THE SAME VENV PATH ISSUE THAT I DO

In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/ng-video-lecture/venv/lib/python3.11/site-packages')

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
from typing import List
import math
import numpy as np
import matplotlib.pyplot as plt

#### !!!! ONLY FOR APPLE SILICON

In [2]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# MatryoshkaGPT

the idea here is to have a bunch of tiny models inside the main model like russian nesting dolls

In [3]:
# hyperparameters
b = 4 # how many independent sequences will we process in parallel?
t = 16 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 50
lr = 3e-4 # learning rate for each backprop step
eval_iters = 20
h = 4 # number of attention heads
l = 4 # number of transormer layers
dropout = 0.2 # % of parameters to ignore every iteration
l2 = 0.01 # multiplier for our L2 norm to encourage sparsity

# embedding aka hidden dimension. this is the largest that th emodel will have
d = 32
power_of_d = int(math.log2(d))
# the smallest power of 2 we'll be considering as a matryoshka embedding
min_power = 3 # Starting from 2^min_power
nesting_list = [2**i for i in range(min_power, int(power_of_d) + 1)]
print(nesting_list)

[8, 16, 32]


In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
v = len(chars)
print(chars, v)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - t, (b,))
    x = torch.stack([data[i:i+t] for i in ix])
    y = torch.stack([data[i+1:i+t+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
# so you can see what the tokenized data looks like
x,y = get_batch('train')
print("x ", x.shape, "\n", x)
print("y ", y.shape, "\n", y)

x  torch.Size([4, 16]) 
 tensor([[51, 39, 56, 56, 63,  1, 46, 43, 56,  6,  1, 47, 44,  1, 46, 43],
        [53, 42, 63,  1, 42, 53, 45,  1, 47, 57,  1, 42, 43, 39, 42,  8],
        [ 1, 21,  1, 49, 52, 53, 61,  6,  1, 54, 43, 58, 47, 58, 47, 53],
        [43,  1, 57, 59, 40, 51, 47, 57, 57, 47, 53, 52,  2,  0, 13, 50]])
y  torch.Size([4, 16]) 
 tensor([[39, 56, 56, 63,  1, 46, 43, 56,  6,  1, 47, 44,  1, 46, 43, 56],
        [42, 63,  1, 42, 53, 45,  1, 47, 57,  1, 42, 43, 39, 42,  8,  0],
        [21,  1, 49, 52, 53, 61,  6,  1, 54, 43, 58, 47, 58, 47, 53, 52],
        [ 1, 57, 59, 40, 51, 47, 57, 57, 47, 53, 52,  2,  0, 13, 50, 50]])


In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

# FEEDFORWARD

chatGPT's attempt at the matryoshkaFeedForward()

supposedly it's more efficient bc my use of .weight on the linear layers was stupid, and it also has the benefit of bringing back bias vectors. 

Honestly yeah it does look better than mine i think i should try it after i confirm that mine bare-minimum functions for sake of my own pride. for the record tho it got to look at mine before making its edits so it's not like it could've understood the concept from scratch
```
class matryoshkaFeedForward_chatGPT(nn.Module):
    def __init__(self, nesting_list, dropout_rate):
        super().__init__()
        
        # The largest embedding dimension of the model
        self.d = nesting_list[-1]

        # Initialize only the largest weights and biases
        self.weight_w1 = nn.Parameter(torch.Tensor(4 * self.d, self.d))
        self.bias_w1 = nn.Parameter(torch.Tensor(4 * self.d))
        self.weight_w2 = nn.Parameter(torch.Tensor(self.d, 4 * self.d))
        self.bias_w2 = nn.Parameter(torch.Tensor(self.d))

        # Initialize weights and biases
        self.reset_parameters()

        self.nesting_list = nesting_list
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout_rate)

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight_w1, a=math.sqrt(5))  # or any other initialization
        nn.init.kaiming_uniform_(self.weight_w2, a=math.sqrt(5))  # or any other initialization
        fan_in1, _ = nn.init._calculate_fan_in_and_fan_out(self.weight_w1)
        bound1 = 1 / math.sqrt(fan_in1)
        nn.init.uniform_(self.bias_w1, -bound1, bound1)
        fan_in2, _ = nn.init._calculate_fan_in_and_fan_out(self.weight_w2)
        bound2 = 1 / math.sqrt(fan_in2)
        nn.init.uniform_(self.bias_w2, -bound2, bound2)

    def forward(self, x_0):
        x_f = ()
        for i, d_i in enumerate(self.nesting_list):
            # Subset the weights and biases
            weight_w1_sub = self.weight_w1[:4 * d_i, :d_i]
            bias_w1_sub = self.bias_w1[:4 * d_i]
            weight_w2_sub = self.weight_w2[:d_i, :4 * d_i]
            bias_w2_sub = self.bias_w2[:d_i]

            # Apply the linear transformations using the subset weights and biases
            x = F.linear(x_0[i], weight_w1_sub, bias_w1_sub)
            x = self.relu(x)
            x = F.linear(x, weight_w2_sub, bias_w2_sub)
            x = self.drop(x)
            x_f += (x,)

        return x_f
```

In [11]:
class matryoshkaFeedFoward(nn.Module):
    def __init__(self, nesting_list: List, dropout):
        super().__init__()
        
        # the largest embedding dimension of the model
        self.d = nesting_list[-1]

        # initialize only the largest. we'll subset later during forward()
        self.w1 = nn.Linear(self.d, 4 * self.d) 
        self.w2 = nn.Linear(4 * self.d, self.d) 

        # to be used for iterating in forward()
        self.nesting_list = nesting_list
        
        # the other parts
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout)
        # so dropout might become an issue
        # dropping out 10% of an 8x8 matrix will have a different effect from dropping out 10% of a 1024x1024 one
        # and potentially more importantly, different weights will get dropped out for each nesting doll
        # this may actually be beneficial in terms of the model's generalizability, but maybe it'll be bad idk
                    
    def forward(self, x_0):
        """
        input: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
        output: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
        """
        # old
        #return self.drop(self.w2(self.relu(self.w1(x))))

        x_f = ()
        for i, d_i in enumerate(self.nesting_list): # i is int from 0 to g-1 while d_i=nesting_list[i]
            x_f += (self.drop(self.relu(x_0[i] @ self.w1.weight[:d_i,:4*d_i]) @ self.w2.weight[:4*d_i,:d_i]),)

        return x_f

# ATTENTION

In [12]:
class matryoshkaHead(nn.Module):
    def __init__(self, nesting_list: List, head_sizes: List):
        super().__init__()
        
        # the largest embedding dimension of the model
        self.d = nesting_list[-1]
        # the largest head size
        self.h = head_sizes[-1]

        # to be used for iterating in forward()
        self.nesting_list = nesting_list
        self.head_sizes = head_sizes

        # initialize only the largest. we'll subset later during forward()
        self.key = nn.Linear(self.d, self.h, bias=False)
        self.query = nn.Linear(self.d, self.h, bias=False)
        self.value = nn.Linear(self.d, self.h, bias=False)

        # the mask so they only look into the past
        self.register_buffer('tril', torch.tril(torch.ones(t, t))) # mask future timestesps

    def forward(self, x_0):
        """
        input: tuple length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
        output: tuple length g with tensors of shape (b,t,h_i) for h_i=head_sizes[i]
        """
        #b,t,d = x.shape
        k,q,wei,out = (),(),[],()
        for i, (d_i, h_i) in enumerate(zip(self.nesting_list, self.head_sizes)):
            k += (x_0[i] @ self.key.weight[:d_i, :h_i],)
            q += (x_0[i] @ self.query.weight[:d_i, :h_i],)
            v += (x_0[i] @ self.value.weight[:d_i, :h_i],)

            # not sure if this is a bunch of "in-place" operations
            # if i get an error about that then what i gotta do is make it separate variables instead of repeatedly editing wei
            wei += q[i] @ k[i].transpose(-2,-1) * k[i].shape[-1]**-0.5
            wei[i] = wei[i].masked_fill(self.tril[:t,:t] == 0, float('-inf'))
            wei[i] = F.softmax(wei[i],dim=-1)
            
            out += (wei[i]@v[i],)

        return out

# -------------------- BOOKMARK -----------------------------------

In [17]:
class matryoshkaMultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, nesting_list: List, head_sizes: List, dropout):
        super().__init__()
        
        self.nesting_list = nesting_list
        self.head_sizes = head_sizes
        self.h_count = len(head_sizes) # number of heads
        self.h = head_sizes[-1] # size of largest head
        self.d = nesting_list[-1]

        # thinking maybe i should mess with the forward() to figure out what i need here
        self.heads = nn.ModuleDict()
        for head_idx in range(h):
            self.heads[f'head_{head_idx}'] = matryoshkaHead(nesting_list, head_sizes)

        # can i even use module list if i'm listing tuples rather than tensors? idk prolly not
        # maybe i can create different module lists using setattr() and selections from the outputs of the heads?
        #self.heads = nn.ModuleList([matryoshkaHead(self.nesting_list, self.head_sizes) for _ in range(self.h)])
        
        #self.proj = nn.Linear(head_sizes[-1] * h, nesting_list[-1])
        # the linear projection that combines the outputs of all the heads
        self.weight_w1 = nn.Parameter(torch.Tensor(self.h * self.h_count, self.h)))
        self.bias_w1 = nn.Parameter(torch.Tensor(self.h * self.h_count))

        # Initialize weights and biases
        self.reset_parameters()
        
        self.dropout = nn.Dropout(dropout)

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight_w1, a=math.sqrt(5))  # or any other initialization
        fan_in1, _ = nn.init._calculate_fan_in_and_fan_out(self.weight_w1)
        bound1 = 1 / math.sqrt(fan_in1)
        nn.init.uniform_(self.bias_w1, -bound1, bound1)
        
    def forward(self, x):
        """
        input: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
            input to each head: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
            output from each head: tuple length g with tensors of shape (b,t,h_i) for h_i=head_sizes[i]
        output: tuple of length g with tensors of shape (b,t,d_i) for d_i=nesting_list[i]
        """
        # i think i gotta create `out` & do selections to make these
        out = [] # do i turn into tuple later or just concatenate?
        for i, (d_i, h_i) in enumerate(zip(nesting_list, head_sizes)):
        
        #out = torch.cat([head(x) for head in self.heads], dim=-1)
        #out = self.dropout(self.proj(out))
        return out

In [ ]:
class matryoshkaMultiHeadAttention(nn.Module):
    def __init__(self, h, nesting_list, head_sizes, dropout=0.1):
        super().__init__()
        
        self.nesting_list = nesting_list
        self.head_sizes = head_sizes
        self.h = h  # number of heads

        # Initialize heads for each combination of granularity and head size
        self.heads = nn.ModuleDict()
        for head_idx in range(h):
            for i, (d_i, h_i) in enumerate(zip(nesting_list, head_sizes)):
                self.heads[f'head_{head_idx}_level_{i}'] = matryoshkaHead([d_i], [h_i])

        # Output transformation layers for each granularity level to ensure output shape consistency
        self.output_transforms = nn.ModuleList([nn.Linear(h_i * h, d_i) for d_i, h_i in zip(nesting_list, head_sizes)])
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_0):
        out = []
        for i, (d_i, h_i) in enumerate(zip(self.nesting_list, self.head_sizes)):
            # Aggregate outputs from all heads for the current granularity level
            head_outputs = []
            for head_idx in range(self.h):
                head = self.heads[f'head_{head_idx}_level_{i}']
                head_output = head((x_0[i],))  # matryoshkaHead expects a tuple input
                head_outputs.append(head_output[0])  # Unpack the single-element tuple

            # Concatenate along the last dimension and apply the output transformation
            combined_output = torch.cat(head_outputs, dim=-1)
            transformed_output = self.output_transforms[i](self.dropout(combined_output))
            out.append(transformed_output)

        # Return a tuple of tensors to maintain consistency with other components
        return tuple(out)


In [ ]:
class matryoshkaMultiHeadAttention(nn.Module):
    def __init__(self, h, nesting_list, dropout):
        super().__init__()
        self.heads = nn.ModuleList([AdaptiveHead(max(nesting_list), nesting_list, dropout) for _ in range(h)])
        self.projections = nn.ModuleDict({
            str(d_i): nn.Linear(d_i * h, d_i) for d_i in nesting_list
        })
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_tuple):
        head_outputs = [head(x_tuple) for head in self.heads]  # List of tuples

        # Concatenate outputs from all heads
        concatenated = tuple(torch.cat([head_output[i] for head_output in head_outputs], dim=-1) for i in range(len(x_tuple)))

        # Project concatenated outputs back to original dimensions
        projected = tuple(self.dropout(self.projections[str(x.size(-1))](concatenated[i])) for i, x in enumerate(x_tuple))

        return projected

chatGPT's rough sketch of the whole attention process

```
class AdaptiveHead(nn.Module):
    def __init__(self, max_head_size, nesting_list):
        super().__init__()
        # Initialize weights for the largest dimension
        self.query_weights = nn.Parameter(torch.Tensor(max_head_size, max_head_size))
        self.key_weights = nn.Parameter(torch.Tensor(max_head_size, max_head_size))
        self.value_weights = nn.Parameter(torch.Tensor(max_head_size, max_head_size))
        self.nesting_list = nesting_list
        # Other initializations (dropout, etc.)

    def forward(self, x_tuple):
        outputs = []
        for x, d_i in zip(x_tuple, self.nesting_list):
            # Adjust weights and operations for d_i
            # Compute attention and add to outputs
            outputs.append(adjusted_output)
        return tuple(outputs)

class MatryoshkaMultiHeadAttention(nn.Module):
    def __init__(self, h, nesting_list):
        super().__init__()
        self.heads = nn.ModuleList([AdaptiveHead(max(nesting_list), nesting_list) for _ in range(h)])
        # Projection layers for each d_i in nesting_list
        self.projections = nn.ModuleDict({str(d_i): nn.Linear(d_i * h, d_i) for d_i in nesting_list})

    def forward(self, x_tuple):
        head_outputs = [head(x_tuple) for head in self.heads]
        # Concatenate, project, and return outputs for each d_i
```

then chatGPT's first attempt at the attention heads
```
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdaptiveHead(nn.Module):
    def __init__(self, max_head_size, nesting_list, dropout):
        super().__init__()
        self.max_head_size = max_head_size
        self.nesting_list = nesting_list
        self.dropout = nn.Dropout(dropout)

        # Parameters for the largest possible head size
        self.query_weights = nn.Parameter(torch.Tensor(max_head_size, max_head_size))
        self.key_weights = nn.Parameter(torch.Tensor(max_head_size, max_head_size))
        self.value_weights = nn.Parameter(torch.Tensor(max_head_size, max_head_size))
        nn.init.normal_(self.query_weights, std=0.02)  # Initializing weights as per common practice
        nn.init.normal_(self.key_weights, std=0.02)
        nn.init.normal_(self.value_weights, std=0.02)

        # Pre-compute masks for efficiency
        self.masks = {}
        for d_i in nesting_list:
            self.masks[d_i] = torch.tril(torch.ones(d_i, d_i)).unsqueeze(0)

    def forward(self, x_tuple):
        outputs = []
        for x, d_i in zip(x_tuple, self.nesting_list):
            b, t, _ = x.size()

            # Slice weights according to the current dimension
            query_w = self.query_weights[:d_i, :d_i]
            key_w = self.key_weights[:d_i, :d_i]
            value_w = self.value_weights[:d_i, :d_i]

            # Compute Q, K, V
            Q = torch.matmul(x, query_w)
            K = torch.matmul(x, key_w)
            V = torch.matmul(x, value_w)

            # Scaled Dot-Product Attention
            attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / (d_i ** 0.5)
            if t <= d_i:  # Ensure the mask fits the time dimension
                mask = self.masks[d_i][:, :t, :t].to(x.device)
                attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))
            attention_probs = F.softmax(attention_scores, dim=-1)
            attention_probs = self.dropout(attention_probs)

            # Weighted sum of values
            out = torch.matmul(attention_probs, V)
            outputs.append(out)

        return tuple(outputs)
```

and then multi-head attention
```
class MatryoshkaMultiHeadAttention(nn.Module):
    def __init__(self, h, nesting_list, dropout):
        super().__init__()
        self.heads = nn.ModuleList([AdaptiveHead(max(nesting_list), nesting_list, dropout) for _ in range(h)])
        self.projections = nn.ModuleDict({
            str(d_i): nn.Linear(d_i * h, d_i) for d_i in nesting_list
        })
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_tuple):
        head_outputs = [head(x_tuple) for head in self.heads]  # List of tuples

        # Concatenate outputs from all heads
        concatenated = tuple(torch.cat([head_output[i] for head_output in head_outputs], dim=-1) for i in range(len(x_tuple)))

        # Project concatenated outputs back to original dimensions
        projected = tuple(self.dropout(self.projections[str(x.size(-1))](concatenated[i])) for i, x in enumerate(x_tuple))

        return projected
```

# BLOCK

In [18]:
class matryoshkaBlock(nn.Module):
    """
    Transformer block: communication followed by computation
    
    input: length g tuple of shape (b,t,d_i) tensors for d_i in nesting_list
    output: length g tuple of shape (b,t,d_i) tensors for d_i in nesting_list
    """

    def __init__(self, h, nesting_list: List):
        # d: the biggest embedding dimension, h: the number of heads we'd like
        super().__init__()

        # originals
        #head_size = d // h # the double backslash just makes the output an int instead of float
        #self.ln = nn.LayerNorm(d, elementwise_affine=False)
        
        self.head_sizes = []
        for i, d_i in enumerate(nesting_list):
            # so we've got a bunch of different head sizes
            self.head_sizes.append(d_i // h)

            # and now we need layernorm attributes for each dimension size
            setattr(self, f"ln_{i}", nn.LayerNorm(d_i, elementwise_affine=False))

        self.mha = MultiHeadAttention(h, head_sizes) #d)
        self.ffwd = FeedFoward(nesting_list) #d)

        # idk if specifying nesting_list is necessary for the forward() but they did it in the original paper
		self.nesting_list = nesting_list
        
    def forward(self, x_i):
        # originals
        #x = x + self.mha(self.ln(x))
        #x = x + self.ffwd(self.ln(x))

        # IF I HAVE TO PIPE IN THE ENTIRE TUPLES
        x_iplus1half = x_i + self.mha
        # ah fuck then i'd have to change the layernorms too




        # IF I CAN PIPE IN INDIVIDUAL TENSORS
        x_iplus1half = ()
        x_iplus1 = ()
        for j in range(len(self.nesting_list)):
            # is this correct? don't i have to pipe in the full x_i into mha and ffwd?
            x_iplus1half[j] = x_i[j] + self.mha(getattr(self, f"ln_{j}")(x_i[j]))
            x_iplus1[j] = x_iplus1half[j] + self.ffwd(getattr(self, f"ln_{j}")(x_iplus1half[j]))
        
        return x_iplus1

TabError: inconsistent use of tabs and spaces in indentation (3307955908.py, line 29)

# OUTPUT

In [19]:
class MRL_Linear_Layer(nn.Module):
    def __init__(self, nesting_list: List, num_classes=v, passed_in_layer=None, **kwargs):
        super(MRL_Linear_Layer, self).__init__()
        self.nesting_list = nesting_list
        self.num_classes = num_classes  # Number of classes for classification
        
        self.passed_in_layer = passed_in_layer  # Store reference to the embedding layer
        self.use_passed_in_layer = passed_in_layer is not None
        if not self.use_passed_in_layer:
            self.nesting_classifier_0 = nn.Linear(nesting_list[-1], self.num_classes, bias=False, device=device, **kwargs)
        
        # Initialize layer normalization
        self.layer_norm = nn.LayerNorm(nesting_list[-1], elementwise_affine=False)

    def forward(self, x):
        if self.use_passed_in_layer:
            # Apply layer normalization on the fly during the forward pass
            normed_passed_in_layer = self.layer_norm(self.passed_in_layer)
            nesting_classifier_0 = normed_passed_in_layer.t().to(x.device)
        else:
            nesting_classifier_0 = self.nesting_classifier_0

        nesting_logits = ()
        for i, num_feat in enumerate(self.nesting_list):
            nesting_logits += (torch.matmul(x[..., :num_feat], nesting_classifier_0[:num_feat, :]),)
        
        return nesting_logits

In [ ]:
class Matryoshka_CE_Loss(nn.Module):
    '''
    Loss function for Matryoshka Representation Learning 
    '''
    def __init__(self, relative_importance: List[float]=None, **kwargs):
        super(Matryoshka_CE_Loss, self).__init__()
        self.criterion = nn.CrossEntropyLoss(**kwargs)
        
        # relative importance shape: [G]
        # this is optional for if you want to weight them differently
        self.relative_importance = relative_importance

    def forward(self, logits, target):
        # logits shape: [g granularities, b batch size, t sequence length, v number of classes]
        # EXCEPT IT COMES IN THE FORM OF A TUPLE OF TENSORS SO REALLY
        g = len(logits)
        b,t,v = logits[-1].shape
        # target shape: [b batch size, t sequence length]

        # Calculate losses for each output and stack them
        # might need to do .view() or .reshape() to make sure these go in well
        losses = torch.stack([self.criterion(logits_i.view(b*t, v), target.view(b*t)) for logits_i in logits])

        # Set relative_importance to 1 if not specified
        # I don't think i'm gonna be emssing around with this part
        rel_importance = torch.ones_like(losses) if self.relative_importance is None else torch.tensor(self.relative_importance)

        # Apply relative importance weights
        weighted_losses = rel_importance * losses
        return weighted_losses.sum()

# THE MODEL

In [20]:
class matryoshkaGPT(nn.Module):
    def __init__(self, nesting_list: List):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(v, d).to(device)
        
        # simple learned positional encodings rather than sine or RoPE
        self.position_embedding_table = nn.Embedding(t, d) 
        self.blocks = nn.Sequential(*[Block(d, h) for _ in range(l)]) # bulk of the beast
        self.ln = nn.LayerNorm(d, elementwise_affine=False) # final layer norm

        # the list of dimensions we'll be using
        self.nesting_list = nesting_list
        
        ### MATRYOSHKA OUTPUT HEADS
        self.m_head = MRL_Linear_Layer(nesting_list = nesting_list, passed_in_layer = self.token_embedding_table.weight)
        
        ### MATRYOSHKA LOSS
        self.m_loss = Matryoshka_CE_Loss()
        
        # initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        b, t = idx.shape
        
        # idx and targets are both (b,t) tensor of integers
        pos_emb = self.position_embedding_table(torch.arange(t, device=device)) # (t,d)
        tok_emb = self.token_embedding_table(idx) # (b,t,d)

        #x = self.ln(tok_emb) + pos_emb # (b,t,d) + (t,d) = (b,t,d)
        # our first nested thingy
        nesting_x0 = ()
        for i, num_feat in enumerate(self.nesting_list):
            nesting_x0 += (self.ln(tok_emb)[...,:num_feat] + pos_emb[...,:num_feat],) # (b,t,d) + (t,d) = (b,t,d)
        # so in total the for loop gives us (b,t,d) & (t,d) -> g*(b,t,d_i) for d_i in nesting_list





        
        #x = self.blocks(x) # (b,t,d) -> (b,t,d)
        # now i need to make the blocks take a tuple of tensors as input
        nexting_x = self.blocks(nesting_x0)




        
        
        x = self.ln(x) # (b,t,d) -> (b,t,d)

        # Matryoshka output head
        logits = self.m_head(x) # tensor [b,t,d] -> tuple ([b,t,d_0], [b,t,d_1], [b,t,d_2],..., [b,t,d_g])
        
        if targets is None:
            loss = None
        else:
            loss = self.m_loss(logits, targets) # g*(b,t,d) & (b,t) -> float

        return logits, loss

# TRAINING

In [ ]:
model = matryoshkaGPT(nesting_list=nesting_list).to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=l2)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

In [ ]:
start_time = time.time()

# Enable anomaly detection
#torch.autograd.set_detect_anomaly(True)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # train
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        current_time = time.time()
        elapsed_time = current_time - start_time
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time elapsed: {elapsed_time:.2f} seconds")

# Disable anomaly detection after the training loop
#torch.autograd.set_detect_anomaly(False)

In [ ]:
## save the trained model
torch.save(model.state_dict(), f'models/{model.__class__.__name__}_b{b}_t{t}_d{d}_h{h}_l{l}_lr{lr}_drop{dropout}_l2-{l2}_min_power{min_power}_{time.strftime("%Y-%m-%d|%H-%M-%S")}.pth')

# Load a saved model

In [ ]:
model = matryoshka GPT().to(device)  # Initialize a model with the same architecture

# Load the saved state dictionary
model.load_state_dict(torch.load('models/GPT_b24_t128_d128_h8_l8_lr0.0003_drop0.2_l2-0.01_2024-01-25|23-31-12.pth'))

# If you plan to continue training the model, switch to training mode
#model.train()

# If you only plan to do inference, switch to evaluation mode
model.eval()

# Inference

In [ ]:
%%time
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R" # the classic line
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
output = model.generate(context_tensor, max_new_tokens=100, degree = min_power)
output_str = decode(output[0].tolist())
print(output_str)

In [ ]:
%%time
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R" # the classic line
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
output = model.generate(context_tensor, max_new_tokens=100, degree = power_of_d)
output_str = decode(output[0].tolist())
print(output_str)